# Maximal Representative Subsampling

In [100]:
from pathlib import Path
import os
import pandas as pd
import numpy as np

path = Path(os.getcwd()).parent


## Attribute Comparison

Each row is labelled with its (survey) *source*.

- *Source*
- 

### Preprocessing: Allensbach

In [120]:
allensbach_cols = ['S01', 'S02', 'S03_rec', 'S04_rec', 'S05',
                   'V04_01', 'V04_02', 'V04_03', 'V04_04', 'V04_05', 'V04_06',
                   'S06b_rec', 'V01', 'V22',
                   'V20_5', 'V20_6', 'V20_7',
                   'V20_1', 'V20_2', 'V20_3', 'V20_4']

'''
nicht in ALLENSBACH:
EINW
Frage 17-19 FB_Neuroenhancement
----------------------------------------------

LO01_01

soziale unterstützung nicht in gbs (OS02-OS04)

ASKU ist in Allensbach 3 Fragen. jeweils 5 antworten.
GBS hat ASKU einmalig 10 Antowrten.	

Berugsgruppe zweimal. Zuordnung ändern.

Wohnortgröße fehlt in GBS.

dm20 und dm42 und dm53 nicht in GBS.
'''


allensbach_names = ['Geschlecht', 'Alter', 'Schulabschluss', 'Berufsgruppe', 'Erwerbstätigkeit',
                    'BRS1', 'BRS2', 'BRS3', 'BRS4', 'BRS5', 'BRS6',
                   'wöchentliche Arbeitszeit', 'Optimismus', 'Pessimismus',
                   'ASKU1', 'ASKU2', 'ASKU3',
                   'IE_int1', 'IE_int2', 'IE_ext1', 'IE_ext2']

allensbach = pd.read_csv(os.path.join(path, 'data/Allensbach/abs_data.csv'),
                         usecols = allensbach_cols,
                         delimiter=';', low_memory=False)

allensbach.replace(',', '.', regex=True, inplace=True)
allensbach.replace(' ', np.nan, regex=True, inplace=True)

allensbach.rename(columns= dict(zip(allensbach_cols, allensbach_names)), inplace=True)

allensbach.head()

,Geschlecht,Alter,Erwerbstätigkeit,Optimismus,BRS1,BRS2,BRS3,BRS4,BRS5,BRS6,...,IE_int2,IE_ext1,IE_ext2,ASKU1,ASKU2,ASKU3,Pessimismus,Schulabschluss,Berufsgruppe,wöchentliche Arbeitszeit
0,2,73,2,7,5,4,1,4,4,5,...,1,5,4,1,1,1,1,2,8,3
1,2,24,1,5,2,4,2,5,2,4,...,2,4,4,2,2,2,3,3,3,3
2,1,51,1,2,2,4,4,5,1,5,...,2,4,4,2,2,3,6,3,3,4
3,1,57,1,6,5,3,5,3,5,3,...,2,4,5,1,2,1,2,5,2,3
4,1,64,2,4,4,4,4,4,4,4,...,3,2,2,5,1,1,4,2,1,3


### Preprocessing: GBS

In [121]:
gbs1 = pd.read_csv(os.path.join(path, 'data/GBS/gbs1.csv'))
gbs2 = pd.read_csv(os.path.join(path, 'data/GBS/gbs2.csv'))

mapping1 = pd.read_csv(os.path.join(path, 'data/GBS/gbs_map1.csv'),
                                    encoding = "ISO-8859-1", delimiter = ';')
mapping2 = pd.read_csv(os.path.join(path, 'data/GBS/gbs_map2.csv'),
                                    encoding = "ISO-8859-1", delimiter = ';')

# Rename 'GBS-CODE' column
key = 'GBS-CODE'
mapping1 = mapping1.rename(columns={'GBS Code': key})
mapping2 = mapping2.rename(columns={'GBS Code': key})
gbs1 = gbs1.rename(columns={'gbs_code': key})
gbs2 = gbs2.rename(columns={'GBS-Code': key})

# Merge GBS "Umfrage Wellen" and create dataframe
gbs = pd.merge(gbs1, gbs2, how='inner', on=key)
gbs = pd.merge(gbs, mapping1, how='left', on=key)
gbs = pd.merge(gbs, mapping2, how='left', on=key)

gbs_cols = ['am01', 'am02_02', 'am14', 'am22', 'am17',
           'br01_01', 'br01_02', 'br01_03', 'br01_04', 'br01_05', 'br01_06',
           'am20_01', 'so01_01', 'so02_01',
           'sw02_01', 'sw02_02', 'sw02_03',
           'ie01_01', 'ie01_02', 'ie01_03', 'ie01_04']

gbs = gbs[gbs_cols]
gbs.rename(columns= dict(zip(gbs_cols, allensbach_names)), inplace=True)

del mapping1, mapping2, gbs1, gbs2

# TODO: Combine matching rows instead of selecting every 2nd.
gbs = gbs.iloc[::2, :]

di1 = {'männlich': 1, "Männlich": 1, 'weiblich': 2, "Weiblich": 2, '[NA] Keine Angabe':np.nan}
gbs.replace({"Geschlecht": di1}, inplace=True)

#gbs['Geburtsjahr'] = [2017-(int)(gj[:4]) if gj is not np.nan else np.nan for gj in gbs.Geburtsjahr]

di2 = {'Hochschulabschluss':5, 
       'Abitur, allgemeine oder fachgebundene Hochschulreife (Gymnasium bzw. EOS)': 4, 
       'Fachhochschulreife (Abschluss einer Fachoberschule)': 4, 
       'Realschulabschluss (Mittlere Reife)': 3,
       'Hauptschulabschluss/Volksschulabschluss': 2,
       'Abschluss Polytechnische Oberschule 10. Klasse (vor 1965: 8. Klasse)': 1, 
       'Anderer Schulabschluss:': 1}

gbs.replace({'Schulabschluss': di2}, inplace=True)

di3 = {
    'Selbstständige(r) Landwirt/-in bzw. Genossenschaftsbauer/-bäuerin':8,
    'Selbstständige (im Handel, im Gastgewerbe, im Handwerk, in der Industrie, der Dienstleistung, auch Ich-AG oder Produktionsgenossenschaft des Handwerks (PGH)-Mitglied)':5,
    'Angestellte(r)':3,
    'Beamter/Beamtin, Richter/-in, Berufssoldat/-in':4,
    'Akademiker/-in in freiem Beruf (Arzt/Ärztin, Rechtsanwalt/-anwältin, Steuerberater/-in u. Ä.)':2,
    '[NA] Keine Angabe':8,
    'keine passende Angabe':8,
    'Ausbildung':8,
    'Arbeiter/-in':3,
    'mithelfende(r) Familienangehörige(r)':1
}
gbs.replace({'Berufsgruppe':di3}, inplace=True)

arbeitszeit = []
for w in gbs['wöchentliche Arbeitszeit']:
    if w < 20:
        arbeitszeit.append(1)
    elif w >= 20 and w < 30:
        arbeitszeit.append(2)
    elif w >= 30 and w <= 40:
        arbeitszeit.append(3)
    elif w > 40 and w < 50:
        arbeitszeit.append(4)
    else:
        arbeitszeit.append(5)
gbs['wöchentliche Arbeitszeit'] = arbeitszeit

di4 = {9:2,
       999:2,
      -3:2,
      -1:2,
      3:2}
# Note: this is also allensbach replacement using same dict
allensbach.replace({'Erwerbstätigkeit': di4}, inplace=True)
allensbach['Erwerbstätigkeit'] = [int(i) for i in allensbach['Erwerbstätigkeit']]
gbs.replace({'Erwerbstätigkeit':di4}, inplace=True)


gbs.head()

,Geschlecht,Alter,Schulabschluss,Berufsgruppe,Erwerbstätigkeit,BRS1,BRS2,BRS3,BRS4,BRS5,...,wöchentliche Arbeitszeit,Optimismus,Pessimismus,ASKU1,ASKU2,ASKU3,IE_int1,IE_int2,IE_ext1,IE_ext2
0,1.0,38.0,4.0,5.0,1.0,4.0,3.0,4.0,3.0,2.0,...,4,3.0,5.0,3.0,2.0,3.0,3.0,2.0,4.0,4.0
2,2.0,71.0,5.0,4.0,1.0,4.0,2.0,4.0,2.0,4.0,...,1,6.0,5.0,3.0,3.0,3.0,5.0,4.0,1.0,3.0
4,1.0,68.0,5.0,8.0,1.0,3.0,2.0,4.0,4.0,2.0,...,1,6.0,3.0,3.0,3.0,3.0,5.0,4.0,2.0,1.0
6,2.0,25.0,5.0,3.0,1.0,4.0,2.0,4.0,1.0,2.0,...,1,6.0,3.0,2.0,3.0,1.0,2.0,4.0,2.0,1.0
8,2.0,33.0,2.0,3.0,1.0,4.0,4.0,4.0,4.0,4.0,...,3,4.0,2.0,4.0,4.0,3.0,5.0,5.0,2.0,2.0


In [123]:

'''
di5 = {'sehr viel Anteilnahme und Interesse':5, 
       'viel Anteilnahme und Interesse': 4, 
       'weder viel noch wenig': 3, 
       'wenig Anteilnahme und Interesse': 2, 
       'keine Anteilnahme und Interesse': 1,
       'keine Angabe':3}
gbs.replace({'Soziale Unterstuetzung': di5}, inplace=True)
'''

"\ndi5 = {'sehr viel Anteilnahme und Interesse':5, \n       'viel Anteilnahme und Interesse': 4, \n       'weder viel noch wenig': 3, \n       'wenig Anteilnahme und Interesse': 2, \n       'keine Anteilnahme und Interesse': 1,\n       'keine Angabe':3}\ngbs.replace({'Soziale Unterstuetzung': di5}, inplace=True)\n"

In [126]:
gbs.columns

Index(['Geschlecht', 'Alter', 'Schulabschluss', 'Berufsgruppe',
       'Erwerbstätigkeit', 'BRS1', 'BRS2', 'BRS3', 'BRS4', 'BRS5', 'BRS6',
       'wöchentliche Arbeitszeit', 'Optimismus', 'Pessimismus', 'ASKU1',
       'ASKU2', 'ASKU3', 'IE_int1', 'IE_int2', 'IE_ext1', 'IE_ext2'],
      dtype='object')

In [ ]:
#+berufstätigkeit

allensbach = allensbach.fillna(allensbach.median()) # quick fix, for the time being.
